<a href="https://colab.research.google.com/github/msc-acse/acse-9-independent-research-project-Garethlomax/blob/full_data_run/full_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#imports


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:

%cd /content/drive/My \Drive/masters_project/python_modules/MovingMNIST-master

# all torch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision.transforms as transforms
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

import matplotlib.pyplot as plt
import h5py

# importing moving mnist test set.
from MovingMNIST import MovingMNIST

%cd /content/drive/My \Drive/masters_project/python_modules/pytorch-summary-master
from torchsummary import summary

# %cd /content/drive/My \Drive/masters_project/python_modules/pytorch_modelsize-master

%cd /content/drive/My \Drive/masters_project/python_modules/pytorchvis-master

!pip install torchviz

%cd /content/drive/My\ Drive/masters_project/python_modules/pytorch-ssim-master
import pytorch_ssim # cite this 


torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True



/content/drive/My Drive/masters_project/python_modules/MovingMNIST-master
/content/drive/My Drive/masters_project/python_modules/pytorch-summary-master
[Errno 2] No such file or directory: '/content/drive/My Drive/masters_project/python_modules/pytorchvis-master'
/content/drive/My Drive/masters_project/python_modules/pytorch-summary-master
     |████████████████████████████████| 51kB 3.7MB/s 
  Created wheel for torchviz: filename=torchviz-0.0.1-cp36-none-any.whl size=3521 sha256=41f5790a737f5d783c2fc2f32e380a32dcce52a0f553bcb97d460bcded87169b
  Stored in directory: /root/.cache/pip/wheels/2a/c2/c5/b8b4d0f7992c735f6db5bfa3c5f354cf36502037ca2b585667
Successfully built torchviz
/content/drive/My Drive/masters_project/python_modules/pytorch-ssim-master


In [3]:
h5py.run_tests()

.....................................................x...................................................................x....................................s...s......ss.......................................................................................................ssssss...................................................................x....x.........................x......x.................................................ssss..................
----------------------------------------------------------------------
Ran 457 tests in 1.548s

OK (skipped=14, expected failures=6)


<unittest.runner.TextTestResult run=457 errors=0 failures=0>

## snippet to investigate ram

In [4]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=6722d1afbf79e7715e0af143d4481e6829faf16d398f107f2b5580cc73866493
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 309.7 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [5]:
%cd /content/drive/My\ Drive/masters_project/data/models

/content/drive/My Drive/masters_project/data/models


## cuda imports

In [6]:

device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    device = 'cuda'
else:
    print("No GPU available!")
    
    
import random
def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = True  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = True

    return True
  
set_seed(42)




Cuda installed! Running on GPU!


True

# LSTM CELL


In [0]:
"""TODO: CUDIFY EVERYTHING"""





class LSTMunit(nn.Module):
    def __init__(self, input_channel_no, hidden_channels_no, kernel_size, stride = 1):
        super(LSTMunit, self).__init__()
        """base unit for an overall convLSTM structure. convLSTM exists in keras but
        not pytorch. LSTMunit repersents one cell in an overall convLSTM encoder decoder format
        the structure of convLSTMs lend themselves well to compartmentalising the LSTM
        cells. 
    
        Each cell takes an input the data at the current timestep Xt, and a hidden
        representation from the previous timestep Ht-1
    
        Each cell outputs Ht
        """
    
    
        self.input_channels = input_channel_no
    
        self.output_channels = hidden_channels_no
    
        self.kernel_size = kernel_size
    
        self.padding = (int((self.kernel_size - 1) / 2 ), int((self.kernel_size - 1) / 2 ))#to ensure output image same dims as input
        # as in conv nowcasting - see references 
        self.stride = stride # for same reasons as above
        
        # need convolutions, cells, tanh, sigmoid?
        # need input size for the lstm - on size of layers.
        # cannot do this because of the modules not being registered when stored in a list
        # can if we convert it to a parameter dict
    
        # list of names of filter to put in dictionary.
        # some of these are not convolutions
        """TODO: CHANGE THIS LAYOUT OF CONVOLUTIONAL LAYERS. """
        
        
        
        self.filter_name_list = ['Wxi', 'Wxf', 'Wxc', 'Wxo','Whi', 'Whf', 'Whc', 'Who']
        
        """ TODO : DEAL WITH BIAS HERE. """ 
        """ TODO: CAN INCLUDE BIAS IN ONE OF THE CONVOLUTIONS BUT NOT ALL OF THEM - OR COULD INCLUDE IN ALL? """

        # list of concolution instances for each lstm cell step
       #  nn.Conv2d(1, 48, kernel_size=3, stride=1, padding=0),
        self.conv_list = [nn.Conv2d(self.input_channels, self.output_channels, kernel_size =  self.kernel_size, stride = self.stride, padding = self.padding, bias = False).cuda() for i in range(4)]
#         self.conv_list = [nn.Conv2d(self.input_channels, self.output_channels, kernel_size =  self.kernel_size, stride = self.stride, padding = self.padding, bias = False) for i in range(4)]

#         self.conv_list = self.conv_list + [(nn.Conv2d(self.output_channels, self.output_channels, kernel_size =  self.kernel_size, stride = self.stride, padding = self.padding, bias = True)).double() for i in range(4)]

        self.conv_list = self.conv_list + [(nn.Conv2d(self.output_channels, self.output_channels, kernel_size =  self.kernel_size, stride = self.stride, padding = self.padding, bias = True).cuda()).double() for i in range(4)]
#         self.conv_list = nn.ModuleList(self.conv_list)
        # stores nicely in dictionary for compact readability.
        # most ML code is uncommented and utterly unreadable. Here we try to avoid this
        self.conv_dict = nn.ModuleDict(zip(self.filter_name_list, self.conv_list))
    
        # may be able to combine all the filters and combine all the things to be convolved - as long as there is no cross layer convolution
        # technically the filter will be the same? - check this later.
    
        # set up W_co, W_cf, W_co as variables.
        """ TODO: decide whether this should be put into function. """
        
        
        """TODO: put correct dimensions of tensor in shape"""
        
        # of dimensions seq length, hidden layers, height, width
        """TODO: DEFINE THESE SYMBOLS. """
        """TODO: PUT THIS IN CONSTRUCTOR."""
        shape = [1, self.output_channels, 16, 16]
        
        self.Wco = nn.Parameter((torch.zeros(shape).double()).cuda(), requires_grad = True)
        self.Wcf = nn.Parameter((torch.zeros(shape).double()).cuda(), requires_grad = True)
        self.Wci = nn.Parameter((torch.zeros(shape).double()).cuda(), requires_grad = True)
        
        
#         self.Wco = nn.Parameter((torch.zeros(shape).double()), requires_grad = True)
#         self.Wcf = nn.Parameter((torch.zeros(shape).double()), requires_grad = True)
#         self.Wci = nn.Parameter((torch.zeros(shape).double()), requires_grad = True)
#         self.Wco.name = "test"
#         self.Wco = torch.zeros(shape, requires_grad = True).double()
#         self.Wcf = torch.zeros(shape, requires_grad = True).double()
#         self.Wci = torch.zeros(shape, requires_grad = True).double()

        # activation functions.
        self.tanh = torch.tanh
        self.sig  = torch.sigmoid

#     (1, 6, kernel_size=5, padding=2, stride=1).double()
    def forward(self, x, h, c):
        """ put the various nets in here - instanciate the other convolutions."""
        """TODO: SORT BIAS OUT HERE"""
        """TODO: PUT THIS IN SELECTOR FUNCTION? SO ONLY PUT IN WXI ECT TO MAKE EASIER TO DEBUG?"""
#         print("size of x is:")
#         print(x.shape)
        # ERROR IS IN LINE 20
        #print(self.conv_dict['Wxi'](x).shape)
#         print("X:")
#         print(x.is_cuda)
#         print("H:")
#         print(h.is_cuda)
#         print("C")
#         print(c.is_cuda)
        
        i_t = self.sig(self.conv_dict['Wxi'](x) + self.conv_dict['Whi'](h) + self.Wci * c)
        f_t = self.sig(self.conv_dict['Wxf'](x) + self.conv_dict['Whf'](h) + self.Wcf * c)
        c_t = f_t * c + i_t * self.tanh(self.conv_dict['Wxc'](x) + self.conv_dict['Whc'](h))
        o_t = self.sig(self.conv_dict['Wxo'](x) + self.conv_dict['Who'](h) + self.Wco * c_t)
        h_t = o_t * self.tanh(c_t)
        
        return h_t, c_t
    
    def copy_in(self):
        """dummy function to copy in the internals of the output in the various architectures i.e encoder decoder format"""

# lstm full unit

In [0]:
"""TODO: IMPORTANT 
WHEN COPYING STATES OVER, INITIAL STATE OF DECODER IS BOTH LAST H AND LAST C 
FROM THE LSTM BEING COPIED FROM.

WE ALSO NEED TO INCLUDE THE ABILITY TO OUTPUT THE LAST H AND C AT EACH TIMESTEP
AS INPUT.
"""


""" SEQUENCE, BATCH SIZE, LAYERS, HEIGHT, WIDTH"""

class LSTMmain(nn.Module):
    
    
    """ collection of units to form encoder/ decoder branches - decide which are which
    need funcitonality to copy in and copy out outputs.
    
    
    layer output is array of booleans selectively outputing for each layer i.e 
    for three layer can have output on second and third but not first with 
    layer_output = [0,1,1]"""
    
    """TODO: DECIDE ON OUTPUT OF HIDDEN CHANNEL LIST """
    def __init__(self, shape, input_channel_no, hidden_channel_no, kernel_size, layer_output, test_input, copy_bool = False, debug = False, save_outputs = True, decoder = False, second_debug = False):
        super(LSTMmain, self).__init__()
        
        """TODO: USE THIS AS BASIS FOR ENCODER DECODER."""
        """TODO: SPECIFY SHAPE OF INPUT VECTOR"""
        
        """TODO: FIGURE OUT HOW TO IMPLEMENT ENCODER DECODER ARCHITECUTRE"""
        self.copy_bool = copy_bool
        
        self.test_input = test_input
        
        self.debug = debug
        self.second_debug = second_debug
        self.save_all_outputs = save_outputs
        
        self.shape = shape
        
        """specify dimensions of shape - as in channel length ect. figure out once put it in a dataloader"""
        
        self.layers = len(test_input) #number of layers in the encoder. 
        
        self.seq_length = shape[1]
        
        self.enc_len = len(shape)
        
        self.input_chans = input_channel_no
        
        self.hidden_chans = hidden_channel_no
        
        self.kernel_size = kernel_size
        
        self.layer_output = layer_output
        
        # initialise the different conv cells. 
#         self.unit_list = [LSTMunit(input_channel_no, hidden_channel_no, kernel_size) for i in range(self.enc_len)]
        self.dummy_list = [input_channel_no] + list(self.test_input) # allows test input to be an array
        if self.debug:
            print("dummy_list:")
            print(self.dummy_list)
            
#         self.unit_list = nn.ModuleList([(LSTMunit(self.dummy_list[i], self.dummy_list[i+1], kernel_size).double()).cuda() for i in range(len(self.test_input))])
        self.unit_list = nn.ModuleList([(LSTMunit(self.dummy_list[i], self.dummy_list[i+1], kernel_size).double()).cuda() for i in range(len(self.test_input))])
        
        if self.debug:
            print("number of units:")
            print(len(self.unit_list))
#             print("number of ")

#         self.unit_list = nn.ModuleList(self.unit_list)
    
    
    def forward(self, x, copy_in = False, copy_out = [False, False, False]):
#     def forward(self, x):
#         copy_in = False
#         copy_out = [False, False, False]

        
#         print("IS X CUDA?")
#         print(x.is_cuda)
        """loop over layers, then over hidden states
        
        copy_in is either False or is [[h,c],[h,c]] ect.
        
        THIS IN NOW CHANGED TO COPY IN 
        
        """
        
        internal_outputs = []
        """TODO: HOW MANY OUTPUTS TO SAVE"""
        """ S """
        
        """ TODO: PUT INITIAL ZERO THROUGH THE SYSTEM TO DEFINE H AND C"""
        
        layer_output = [] # empty list to save each h and c for each step. 
        """TODO: DECIDE WHETHER THE ABOVE SHOULD BE ARRAY OR NOT"""
        
        # x is 5th dimensional tensor.
        # x is of size batch, sequence, layers, height, width
        
        """TODO: INITIALISE THESE WITH VECTORS."""
        # these need to be of dimensions (batchsizze, hidden_dim, heigh, width)
        
        size = x.shape
        
        # need to re arrange the outputs. 
        
        
        """TODO: SORT OUT H SIZING. """
        
        batch_size = size[0]
        # change this. h should be of dimensions hidden size, hidden size.
        h_shape = list(x.shape[:1] + x.shape[2:]) # seq is second, we miss it with fancy indexing
        h_shape[1] = self.hidden_chans
        if self.debug:
            print("h_shape:")
            print(h_shape)
        
        # size should be (seq, batch_size, layers, height, weight)
        
        
        empty_start_vectors = []
        
        
        #### new method of copying vectors. copy_bool, assigned during object 
        # construction now deals iwth copying in values.
        # copy in is still used to supply the tensor values. 
    
        k = 0 # to count through our input state list.
        for i in range(self.layers):
            if self.copy_bool[i]: # if copy bool is true for this layer
                # check purpose of h_shape in below code.
                empty_start_vectors.append(copy_in[k])
                # copies in state for that layer
                """TODO: CHECK IF THIS NEEDS TO BE DETATCHED OR NOT"""
                k += 1 # iterate through input list.
            
            else: # i.e if false
                assert self.copy_bool[i] == False, "copy_bool arent bools"
                
                h_shape = list(x.shape[:1] + x.shape[2:]) # seq is second, we miss it with fancy indexing
                h_shape[1] = self.dummy_list[i+1] # check indexing. 
                empty_start_vectors.append([(torch.zeros(h_shape).double()).cuda(), (torch.zeros(h_shape).double()).cuda()])
                
        del k # clear up k so no spare variables flying about.
        
        
        
        
#         for i in range(self.layers):
#             """CHANGED: NOW HAS COPY IN COPY OUT BASED ON [[0,0][H,C]] FORMAT"""
#             if copy_in == False: # i.e if no copying in occurs then proceed as normal
#                 h_shape = list(x.shape[:1] + x.shape[2:]) # seq is second, we miss it with fancy indexing
#                 h_shape[1] = self.dummy_list[i+1] # check indexing. 
# #                 empty_start_vectors.append([(torch.zeros(h_shape).double()).cuda(), (torch.zeros(h_shape).double()).cuda()])
#                 empty_start_vectors.append([(torch.zeros(h_shape).double()).cuda(), (torch.zeros(h_shape).double()).cuda()])
# #             elif copy_in[i] == [0,0]:
#             elif isinstance(copy_in[i], list):

#                 assert (len(copy_in) == self.layers), "Length disparity between layers, copy in format"

#                 # if no copying in in alternate format
#                 h_shape = list(x.shape[:1] + x.shape[2:]) # seq is second, we miss it with fancy indexing
#                 h_shape[1] = self.dummy_list[i+1] # check indexing. 
#                 empty_start_vectors.append([(torch.zeros(h_shape).double()).cuda(), (torch.zeros(h_shape).double()).cuda()])
                
#             else: # copy in the provided vectors
#                 assert (len(copy_in) == self.layers), "Length disparity between layers, copy in format"

#                 """TODO: DECIDE WHETHER TO CHANGE THIS TO AN ASSERT BASED OFF TYPE OF TENSOR."""
#                 empty_start_vectors.append(copy_in[i])
                
            
        
        
        
#         empty_start_vectors = [[torch.zeros(h_shape), torch.zeros(h_shape)] for i in range(self.layers)]
        
        
        
        if self.debug:
            for i in empty_start_vectors:
                print(i[0].shape)
            print(" \n \n \n")
        
#         for i in range(self.layers):
#             empty_start_vectors.append([torch.tensor()])
        
        total_outputs = []
        
        
        for i in range(self.layers):
            
            
            layer_output = []
            if self.debug:
                print("layer iteration:")
                print(i)
            # for each in layer

            """AS WE PUT IN ZEROS EACH TIME THIS MAKES OUR LSTM STATELESS"""
            # initialise with zero or noisy vectors 
            # at start of each layer put noisy vector in 
            # look at tricks paper to find more effective ideas of how to put this in
            # do we have to initialise with 0 tensors after we go to the second layer
            # or does the h carry over???
            """TODO: REVIEW THIS CHANGE"""
            
            # copy in for each layer. 
            # this is used for encoder decoder architectures.
            # default is to put in empty vectors. 
            
            """TODO: REVIEW THIS SECTION"""
            """CHANGED: TO ALWAYS CHOOSE H AND C"""
#             if copy_in == False:
#                 h, c = empty_start_vectors[i]
#             else: h, c = copy_in[i]

            h, c = empty_start_vectors[i] 
                
            if self.debug:
                print("new h shape")
                print(h.shape)
                
            """TODO: DO WE HAVE TO PUT BLANK VECTORS IN AT EACH TIMESTEP?"""
            
            # need to initialise zero states for c and h. 
            for j in range(self.seq_length):
                if self.debug:
                    print("inner loop iteration:")
                    print(j)
                if self.debug:
                    print("x dtype is:" , x.dtype)
                # for each step in the sequence
                # put x through 
                # i.e put through each x value at a given time.
                
                """TODO: PUT H IN FROM PREVIOUS LAYER, BUT C SHOULD BE ZEROS AT START"""
                
                if self.debug:
                    print("inner loop size:")
                    print(x[:,j].shape)
                    print("h size:")
                    print(h.shape)
                    
                h, c = self.unit_list[i](x[:,j], h, c)
                
                # this is record for each output in given layer.
                # this depends whether copying out it enabld 
#                 i
                layer_output.append([h, c])
                
            """TODO: IMPLEMENT THIS"""
#             if self.save_all_outputs[i]:
#                 total_outputs.append(layer_outputs[:,0]) # saves h from each of the layer outputs
                
            # output 
            """OUTSIDE OF SEQ LOOP"""
            """TODO: CHANGE TO NEW OUTPUT METHOD."""
            if copy_out[i] == True:
                # if we want to copy out the contents of this layer:
                internal_outputs.append(layer_output[-1])
                # saves last state and memory which can be subsequently unrolled.
                # when used in an encoder decoder format.
            """removed else statement"""
#             else:
#                 internal_outputs.append([0,0])
                # saves null variable so we can check whats being sent out.
            
            
            h_output = [i[0] for i in layer_output] #layer_output[:,0] # take h from each timestep.
            if self.debug:
                print("h_output is of size:")
                print(h_output[0].shape)
                
                      
            """TODO: REVIEW IF 1 IS THE CORRECT AXIS TO CONCATENATE THE VECTORS ALONG"""
            # we now use h as the predictor input to the other layers.
            """TODO: STACK TENSORS ALONG NEW AXIS. """
            
            
            x = torch.stack(h_output,0)
            x = torch.transpose(x, 0, 1)
            if self.second_debug:
                print("x shape in LSTM main:" , x.shape)
            if self.debug:
                print("x reshaped dimensions:")
                print(x.shape)
        
#         x = torch.zeros(x.shape)
#         x.requires_grad = True
        return x , internal_outputs # return new h in tensor form. do we need to cudify this stuff

    def initialise(self):
        """put through zeros to start everything"""
        

# lstm enc dec onestep

In [0]:
# test2 = LSTMmain(shape, 1, 3, 5, [1], test_input = [1,2], debug = False).double()



class LSTMencdec_onestep(nn.Module):
    """structure is overall architecture of """
    def __init__(self, structure, input_channels, kernel_size = 5, debug = True):
        super(LSTMencdec_onestep, self).__init__()
#         assert isinstance(structure, np.array), "structure should be a 2d numpy array"
        assert len(structure.shape) == 2, "structure should be a 2d numpy array with two rows"
        self.debug = debug
        
        """TODO: MAKE KERNEL SIZE A LIST SO CAN SPECIFY AT EACH JUNCTURE."""
        shape = [1,10,3,16,16]
        
        self.structure = structure
        """STRUCTURE IS AN ARRAY - CANNOT USE [] + [] LIST CONCATENATION - WAS ADDING ONE ONTO THE ARRAY THING."""
        self.input_channels = input_channels
        self.kernel_size = kernel_size
        
        """TODO: ASSERT THAT DATATYPE IS INT."""
        
        self.enc_shape, self.dec_shape, self.enc_copy_out, self.dec_copy_in = self.input_test()
        
        if self.debug:
            print("enc_shape, dec_shape, enc_copy_out, dec_copy_in:")
            print(self.enc_shape)
            print(self.dec_shape)
            print(self.enc_copy_out)
            print(self.dec_copy_in)
            
        
        
#         self.sig = nn.Sigmoid()
        
         # why does this have +1 at third input and decoder hasnt?????? 
        
        self.encoder = LSTMmain(shape, self.input_channels, len(self.enc_shape)+1, self.kernel_size, layer_output = self.enc_copy_out, test_input = self.enc_shape, copy_bool = [False for k in range(len(self.enc_shape))]  ).cuda()
        # now one step in sequence
        shape = [1,1,1,64,64]

        self.decoder = LSTMmain(shape, self.enc_shape[-1], len(self.dec_shape), self.kernel_size, layer_output = 1, test_input = self.dec_shape, copy_bool = self.dec_copy_in,  second_debug = False).cuda()
        
        
        
        # initialise encoder and decoder network
    
    def input_test(self):
        """check input structure to make sure there is overlap between encoder 
        and decoder.
        """
        copy_grid = []
        # finds dimensions of the encoder
        enc_layer = self.structure[0]
        enc_shape = enc_layer[enc_layer!=0]
        dec_layer = self.structure[1]
        dec_shape = dec_layer[dec_layer!=0]
#         
        
        
        
        
        
        #set up boolean grid of where the overlaps are.
        for i in range(len(enc_layer)):
            if self.debug:
                print(enc_layer[i], dec_layer[i])
            if (enc_layer[i] != 0) and (dec_layer[i] != 0):
                copy_grid.append(True)
            else:
                copy_grid.append(False)
                
                
        enc_overlap = copy_grid[:len(enc_layer)-1]
        
        num_dec_zeros = len(dec_layer[dec_layer==0]) # will this break if no zeros?
        
        dec_overlap = copy_grid[num_dec_zeros:]
        
        return enc_shape, dec_shape, enc_overlap, dec_overlap
        
#         dec_overlap = copy_grid[]                
        
                
                
#         [[1,2,3,0],
#          [0,2,3,1]]
    def forward(self, x):
        
        x, out_states = self.encoder(x, copy_in = False, copy_out = self.enc_copy_out)
        
#         print("length of out_states:", len(out_states))
#         print("contents out outstates are as follows:")
#         for i in out_states:
#             print("----------------------------------")
#             print("first object type:", type(i[0]))
# #             print("length of object:", len(i[0]))
            
        
        
        
        dummy_input = torch.zeros(x.shape)
        # technically a conditional loader - put x in there 
        # puts in the last one as input - should make shorter. 
        # presume coming out in the correct order - next try reversing to see if that helps 
        x = x[:,-1:,:,:,:]
#         print("x shape encoder:", x.shape)
#         print(x.shape)
        
        
        res, _ = self.decoder(x, copy_in = out_states, copy_out = [False, False, False,False, False])
        print("FINISHING ONE PASS")
#         res = self.sig(res)
        return res

# dataset

In [0]:
class HDF5Dataset(Dataset):
    """dataset wrapper for hdf5 dataset to allow for lazy loading of data. This 
    allows ram to be conserved. 
    
    As the hdf5 dataset is not partitioned into test and validation, the dataset 
    takes a shuffled list of indices to allow specification of training and 
    validation sets.
    
    MAKE SURE TO CALL DEL ON GENERATED OBJECTS OTHERWISE WE WILL CLOG UP RAM
    
    """
    
    def __init__(self, path, index_map, transform = None):
        
        %cd /content/drive/My \Drive/masters_project/data 
        # changes directory to the one where needed.
        
        self.path = path
        
        self.index_map = index_map # maps to the index in the validation split
        # due to hdf5 lazy loading index map must be in ascending order.
        # this may be an issue as we should shuffle our dataset.
        # this will be raised as an issue as we consider a work around.
        # we should keep index map shuffled, and take the selection from the 
        # shuffled map and select in ascending order. 
        
        
        self.file = h5py.File(path, 'r')
        
        
        
        
        
        
    def __len__(self):
        return len(self.index_map)
    
    def __getitem__(self,i):
        
        i = self.index_map[i] # index maps from validation set to select new orders
#         print(i)
        if isinstance(i, list): # if i is a list. 
            i.sort() # sorts into ascending order as specified above
            
        """TODO: CHECK IF THIS RETURNS DOUBLE"""
        
        predictor = torch.tensor(self.file["predictor"][i])
        
        truth = torch.tensor(self.file["truth"][i])
        
        return predictor, truth

In [0]:
def initialise_dataset_HDF5(valid_frac = 0.1, dataset_length = 9000):
    """
    Returns datasets for training and validation. 
    
    Loads in datasets segmenting for validation fractions.
   
    
    
    """
    
    if valid_frac != 0:
        
        dummy = np.array(range(dataset_length)) # clean this up - not really needed
        
        train_index, valid_index = validation_split(dummy, n_splits = 1, valid_fraction = 0.1, random_state = 0)
        
        train_dataset = HDF5Dataset("train_set.hdf5", index_map = train_index)
        
        valid_dataset = HDF5Dataset("test_set.hdf5", index_map = valid_index)
        
        return train_dataset, valid_dataset
        
    else:
        print("not a valid fraction for validation") # turn this into an assert.
        
        
        

In [0]:
def initialise_dataset_HDF5_full(valid_frac = 0.1, dataset_length = 9000):
    """
    Returns datasets for training and validation. 
    
    Loads in datasets segmenting for validation fractions.
   
    
    
    """
    
    if valid_frac != 0:
        
        dummy = np.array(range(dataset_length)) # clean this up - not really needed
        
        train_index, valid_index = validation_split(dummy, n_splits = 1, valid_fraction = 0.1, random_state = 0)
        
        train_dataset = HDF5Dataset("train_set.hdf5", index_map = train_index)
        
        valid_dataset = HDF5Dataset("test_set.hdf5", index_map = valid_index)
        
        return train_dataset, valid_dataset
        
    else:
        print("not a valid fraction for validation") # turn this into an assert.
        

# shuffling functions

In [0]:
def validation_split(data, n_splits = 1, valid_fraction = 0.1, random_state = 0):
    """
    Function to produce a validation set from test set.
    THIS SHUFFLES THE SAMPLES. __NOT__ THE SEQUENCES.
    """
    dummy_array = np.zeros(len(data))
    split = StratifiedShuffleSplit(n_splits, test_size = valid_fraction, random_state = 0)
    generator = split.split(torch.tensor(dummy_array), torch.tensor(dummy_array))
    return [(a,b) for a, b in generator][0]

In [0]:
def unsqueeze_data(data):
    """
    Takes in moving MNIST object - must then account for 
    """
    
    # split moving mnist data into predictor and ground truth.
    predictor = data[:][0].unsqueeze(2)
    predictor = predictor.double()
        
    truth = data[:][1].unsqueeze(2)# this should be the moving mnist sent in
    truth = truth.double()
    
    return predictor, truth
    # the data should now be unsqueezed.

In [0]:
def initialise_dataset(data):
    # unsqueeze data, adding a channel dimension for later convolution. 
    # this also gets rid of the annoying tuple format
    predictor, truth = unsqueeze_data(data)
    
    train_index, valid_index = validation_split(data)
    
    train_predictor = predictor[train_index]
    valid_predictor = predictor[valid_index]
    
    train_truth = truth[train_index]
    valid_truth = truth[valid_index]
    
    train_dataset = SequenceDataset(train_predictor, train_truth)
    valid_dataset = SequenceDataset(valid_predictor, valid_truth)
    
    return train_dataset, valid_dataset

#TRAINING FUNCTIONS


In [0]:
def comb_loss_func(pred, y):
    """hopefully should work like kl and bce for VAE"""
    mse = nn.MSELoss()
    ssim = pytorch_ssim.SSIM()
    mse_loss = mse(pred, y[:,:1,:,:,:])
    ssim_loss = -ssim(pred[:,0,:,:,:], y[:,0,:,:,:])
    return mse_loss + ssim_loss

In [17]:
%cd /content/drive/My\ Drive/masters_project/data/models
def train_enc_dec(model, optimizer, dataloader, loss_func = nn.MSELoss()):
    """
    training function 
    
    by default mseloss
    
    could try brier score.
    
    """
    i = 0
    model.train() # enables training for model. 
    tot_loss = 0
    for x, y in dataloader:
#         print("training")
        x = x.to(device) # send to cuda.
        y = y.to(device)
        optimizer.zero_grad() # zeros saved gradients in the optimizer.
        # prevents multiple stacking of gradients
        # this is important to do before we evaluate the model as the 
        # model is currenly in model.train() mode
        
        prediction = model(x) #x should be properly formatted - of size
        """THIS DOESNT DEAL WITH SEQUENCE LENGTH VARIANCE OF PREDICTION OR Y"""
        
#         print("the size of prediction is:", prediction.shape)
        #last image sequence.
    
        """ACTUAL FUNCTION THATS BEEN COMMENTED OUT."""
#         loss = loss_func(prediction, y[:,:1,:,:,:])
        """CHANGED BECAUSE """
        print(prediction.shape)
        print(y.shape)
        loss = loss_func(prediction[:,0,0], y)
        

#         loss = comb_loss_func(prediction, y)
#         print(prediction.shape)
#         print(y[:,:1,:,:,:].shape)
        """commented out """
#         loss = - loss_func(prediction[:,0,:,:,:], y[:,0,:,:,:])
    
# ssim_out = -ssim_loss(train[0][0][-1:],  x[0])
# ssim_value = - ssim_out.data
    
    
        
        loss.backward() # differentiates to find minimum.
#         printm()

        ##
        # implement the interpreteable stuff here.
        # as it is very unlikely we predict every pixel correctly we will not 
        # use accuracy. 
        # technically this is a regression problem, not a classification.
        
        
        optimizer.step() # steps forward the optimizer.
        # uses loss.backward() to give gradient. 
        # loss is negative.
#         del x # make sure the garbage is collected.
#         del y
        """commented it out"""
        tot_loss += loss.item() # .data.item() 
        print("BATCH:")
        print(i)
        i += 1
#         if i == 20:
#             break
        print("MSE_LOSS:", tot_loss / i)
    return model, tot_loss # trainloss, trainaccuracy 

def validate(model, dataloader, criterion = nn.MSELoss()):
    """as for train_enc_dec but without training - and acting upon validation
    data set
    """
    model.eval() # puts out of train mode so we do not mess up our gradients
    for x, y in dataloader:
        with torch.no_grad: # no longer have to specify tensors 
            # as volatile = True. as of modern pytorch use torch.no_grad.
            
            x.to(device) # send to cuda.
            y.to(device)
            prediction = model(x)
            
            loss = loss_func(prediction, y)
            
            
    return validloss, validaccuracy





def train_main(model, params, train, valid, epochs = 30, batch_size = 1):
    # make sure model is ported to cuda
    # make sure seed has been specified if testing comparative approaches
    
#     if model.is_cuda == False:
#         model.to(device)
    
    # initialise optimizer on model parameters 
    # chann
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.005, amsgrad= True)
    loss_func = nn.MSELoss()
#     loss_func = nn.BCELoss()
#     loss_func = pytorch_ssim.SSIM()
    
    train_loader = DataLoader(train, batch_size = batch_size, shuffle = True) # implement moving MNIST data input
    validation_loader = DataLoader(valid, batch_size = batch_size, shuffle = False) # implement moving MNIST
    
    for epoch in range(epochs):
        
        train_enc_dec(model, optimizer, train_loader, loss_func = loss_func) # changed
        
        
        torch.save(optimizer.state_dict(), F"Adam_new_ams_changed"+str(epoch)+".pth")
        torch.save(model.state_dict(), F"Test_new_ams_changed"+str(epoch)+".pth")
#         validate(model, validation_loader)
        
    return model, optimizer
        
    
    
    
            
            
            
            
    


        
        
        
        
        
        
        
        
        
        
        
        

/content/drive/My Drive/masters_project/data/models


# hdf5 with avgs

In [0]:
class HDF5Dataset_with_avgs(Dataset):
    """dataset wrapper for hdf5 dataset to allow for lazy loading of data. This 
    allows ram to be conserved. 
    
    As the hdf5 dataset is not partitioned into test and validation, the dataset 
    takes a shuffled list of indices to allow specification of training and 
    validation sets.
    
    MAKE SURE TO CALL DEL ON GENERATED OBJECTS OTHERWISE WE WILL CLOG UP RAM
    
    """
    
    def __init__(self, path, index_map, avg, std, application_boolean, transform = None):
        
        %cd /content/drive/My \Drive/masters_project/data 
        # changes directory to the one where needed.
        
        self.path = path
        
        self.index_map = index_map # maps to the index in the validation split
        # due to hdf5 lazy loading index map must be in ascending order.
        # this may be an issue as we should shuffle our dataset.
        # this will be raised as an issue as we consider a work around.
        # we should keep index map shuffled, and take the selection from the 
        # shuffled map and select in ascending order. 
        self.avg = avg
        self.std = std
        self.application_boolean = application_boolean
        
        self.file = h5py.File(path, 'r')
        
#         for i in range(len(application_boolean)):
#             # i.e gaussian transformation doesnt happen. (x - mu / sigma)
#             if application_boolean == 0:
#                 self.avg[i] = 0
#                 self.std[i] = 1
        
        
         
          
        
        
        
        
        
        
    def __len__(self):
        return len(self.index_map)
    
    def __getitem__(self,i):
        
        i = self.index_map[i] # index maps from validation set to select new orders
#         print(i)
        if isinstance(i, list): # if i is a list. 
            i.sort() # sorts into ascending order as specified above
            
        """TODO: CHECK IF THIS RETURNS DOUBLE"""
        
        predictor = torch.tensor(self.file["predictor"][i])
        print("predictor shape:", predictor.shape)
        # is of batch size, seq length, 
        
        
        truth = torch.tensor(self.file["truth"][i])
        print("truth shape:", truth.shape)
        # only on layer so not in loop.
        truth -= self.avg[0]
        truth /= self.std[0]
        
        if isinstance(i, list):
            for j in range(len(self.avg)):
                if self.application_boolean[j]:
                    predictor[:,:,j] -= self.avg[j]
                    predictor[:,:,j] /= self.std[j]
                
                
        else:
            for j in range(len(self.avg)):
                if self.application_boolean[j]:
                    predictor[:,j] -= self.avg[j]
                    predictor[:,j] /= self.std[j]
                
            
#             #i.e if we are returning a single index.
# #         # the value of truth should be [0] in the predictor array. 
#         for j in range(len(self.avg)):
#             if self.application_boolean[j]:
#                 predictor[:,:,j] -= self.avg[j]
#                 predictor[:,:,j] /= self.std[j]
                
#                 # sort out dimensions of truth at some point 
        
        
                
            
        
        return predictor, truth

# wrapper

In [0]:
def wrapper_full(name, optimizer,  structure, loss_func, avg, std, application_boolean, lr = None, epochs = 50, kernel_size = 3, batch_size = 50):
    f = open(name + ".csv", 'w') # open csv file for saving
    
    model = LSTMencdec_onestep(structure, 5, kernel_size = kernel_size).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr = lr, amsgrad= True)

    # put in structure.
    f.write("Structure: \n")

    for i in range(len(structure)):
        for j in range(len(structure[0])):
            f.write("{},".format(structure[i,j]))
        f.write("\n") # new line

    f.write("Parameters:\n")
    f.write("optimizer, epochs, learning rate, kernel size \n")
    
    if lr != None:
        # optimizer problems
        
        f.write("{},{},{},{},{}\n".format("test", epochs, lr, kernel_size, batch_size))
    else:
        f.write("{},{},{},{},{}\n".format("othertest", epochs, "Default", kernel_size, batch_size))
        
    f.write("loss_func:\n")
    f.write(loss_func.__repr__() + "\n")
    
    f.write("optimizer:\n")
    f.write(optimizer.__repr__() + "\n")
    
    f.write("\n\n\n")
    f.write("TRAINING\n")
    
    # now we define the training functions
#     train, valid = initialise_dataset_HDF5()
    
    index_map = np.arange(0, 84485,1)# dummy index_map just to make sure trains correctly. 
    train = HDF5Dataset_with_avgs('data_prio_run_test4.hdf5', index_map,avg, std, apbln)
    
    # put model together with different size
    # model test - put it together.
    # now we train the model
    
    
    
    # ignore this one.
    
#     model, optimizer = train_main(test_model, 1, train, valid, epochs = epochs, batch_size = 50)

    loss_func = loss_func
    
#     optimizer = torch.optim.Adam(model.parameters(), lr = lr, amsgrad= True)
    
#     loss_func = nn.BCELoss()
#     loss_func = pytorch_ssim.SSIM()
    f.close()
    
    train_loader = DataLoader(train, batch_size = batch_size, shuffle = True) # implement moving MNIST data input
#     validation_loader = DataLoader(valid, batch_size = batch_size, shuffle = False) # implement moving MNIST
    
    for epoch in range(epochs):
        
        # does is matter that we arent returning the model?
        _, loss = train_enc_dec(model, optimizer, train_loader, loss_func = loss_func) # changed
        
        f = open(name + ".csv", 'a') # open csv file for saving

        f.write(str(loss) + "\n")
        
        f.close()
        
        torch.save(optimizer.state_dict(), name+str(epoch)+".pth")
        torch.save(model.state_dict(), name+str(epoch)+".pth")


    
    
    
    
#     f.close()

# RUN CODE 

In [0]:
# changed 
structure = np.array([[24,48,0,0,0],[0,48,24,12,5]])

# test_model = LSTMencdec_onestep(structure, 1, kernel_size = 3).to(device)
# print("seq length", test_model.decoder.seq_length)
# optim = torch.optim.Adam(test_model.parameters())



# train_enc = train_enc_dec(test_model,)


# train_main(test_model, 1, train, valid, epochs = 2, batch_size = 50)

# model, optimizer = train_main(test_model, 1, train, valid, epochs = 50, batch_size = 50)


# torch.save(optimizer.state_dict(), F"Finished_opt_bce.pth")
# torch.save(model.state_dict(), F"Finished_mod_bce.pth")



In [0]:
test1 = np.zeros((50,1,1,16,16))

In [22]:
test1[:,0,0].shape


(50, 16, 16)

In [23]:
%pwd


'/content/drive/My Drive/masters_project/data/models'

# trying to unfuck averaging 

todo - put simple occurance of conflict as prediction - occurance of massive events - way bigger than average lead to issues with non gaussian distribution of event sizes. 
use binary to predict.
re run dataset creation tonight and add single binary prediction
- powerlaw transformation?
- double check what the loss function is comparing. 
- make example where it imshows just for fun so can see what the inputs are.


In [27]:
%pwd
%cd ..

/content/drive/My Drive/masters_project/data


In [0]:
avg = np.load("dset4_avg.npy")
std = np.load("dset4_std.npy")
apbln = [1,0,0,0,1] # think this is correct

In [0]:
std

In [31]:
# %cd ..
index_map = np.arange(0,84485,1)
train = HDF5Dataset_with_avgs('data_prio_run_test4.hdf5', list(index_map),avg, std, apbln)

/content/drive/My Drive/masters_project/data


In [32]:
pred = train[0:200][0]

tru = train[0:200][1] 

predictor shape: torch.Size([200, 10, 5, 16, 16])
truth shape: torch.Size([200, 16, 16])
predictor shape: torch.Size([200, 10, 5, 16, 16])
truth shape: torch.Size([200, 16, 16])


In [0]:
pred.shape

In [0]:
tru.shape

In [0]:
b = nn.MSELoss()

In [51]:
-4.4e-2 == -0.044

True

In [59]:
# event 164 has massive mseloss due to large error
pred[164][-1][0].unique(return_counts = True) # why not lots of -0.44 ect 
# plt.imshow(pred[164][-1][0])


(tensor([-0.0440,  0.0822], dtype=torch.float64), tensor([255,   1]))

In [60]:
tru[164].unique(return_counts = True) 
# pred[164][-1][0].shape


# plt.imshow(tru[164])

(tensor([-4.4027e-02,  3.3456e-01,  1.7227e+00,  2.2711e+02],
        dtype=torch.float64), tensor([253,   1,   1,   1]))

In [57]:
b(pred[164][-1][0],tru[164])

tensor(201.5630, dtype=torch.float64)

In [35]:
tot = 0
for i in range(200):
    print(i)
    print(b(pred[i][-1][0], tru[i]))

0
tensor(0.1703, dtype=torch.float64)
1
tensor(0.0004, dtype=torch.float64)
2
tensor(0.0002, dtype=torch.float64)
3
tensor(6.2207e-05, dtype=torch.float64)
4
tensor(0.0002, dtype=torch.float64)
5
tensor(0.0001, dtype=torch.float64)
6
tensor(0.2046, dtype=torch.float64)
7
tensor(0.0002, dtype=torch.float64)
8
tensor(0.0002, dtype=torch.float64)
9
tensor(0.0364, dtype=torch.float64)
10
tensor(0.2119, dtype=torch.float64)
11
tensor(0.4460, dtype=torch.float64)
12
tensor(0.4458, dtype=torch.float64)
13
tensor(0.4536, dtype=torch.float64)
14
tensor(0.4460, dtype=torch.float64)
15
tensor(0.0002, dtype=torch.float64)
16
tensor(0.0001, dtype=torch.float64)
17
tensor(0.2048, dtype=torch.float64)
18
tensor(0.0003, dtype=torch.float64)
19
tensor(0.0002, dtype=torch.float64)
20
tensor(0.2122, dtype=torch.float64)
21
tensor(0.2047, dtype=torch.float64)
22
tensor(0.2044, dtype=torch.float64)
23
tensor(0.2044, dtype=torch.float64)
24
tensor(0.2119, dtype=torch.float64)
25
tensor(0.2122, dtype=torch.f

In [0]:
plt.imshow(pred[0][-1][0])

In [0]:
plt.imshow(tru[0])

In [0]:
plt.imshow(tru)

In [0]:
tru

In [0]:
b(pred[0][0], tru)

In [0]:
# d = train[20000]
d = train[20000:20010]
print(d[0].shape)
plt.imshow(d[1][0])
o = d[0][0]
for i in range(10):
    plt.figure()
    plt.imshow(o[i][0])
    

In [0]:
%pwd
%cd data

In [0]:
%cd 
b = nn.MSELoss()

avg = np.load("dset4_avg.npy")
std = np.load("dset4_std.npy")
apbln = [1,0,0,0,1] # think this is correct
    
wrapper_full("first_run_full_data", 10, structure, b, avg, std, apbln, lr = 0.005, epochs = 200, batch_size = 200)